In [1]:
import pickle
import numpy as np
import pandas as pd
from datetime import datetime
import os

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
import keras_tuner as kt
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint, EarlyStopping

from sklearn import metrics
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.preprocessing import LabelEncoder

In [2]:
with open(r'.\News_Dataset_Splits\X_train.pkl', 'rb') as f:
    X_train = pickle.load(f)

with open(r'.\News_Dataset_Splits\X_val.pkl', 'rb') as f:
    X_val = pickle.load(f)

with open(r'.\News_Dataset_Splits\X_test.pkl', 'rb') as f:
    X_test = pickle.load(f)

with open(r'.\News_Dataset_Splits\y_train.pkl', 'rb') as f:
    y_train = pickle.load(f)

with open(r'.\News_Dataset_Splits\y_val.pkl', 'rb') as f:
    y_val = pickle.load(f)

with open(r'.\News_Dataset_Splits\y_test.pkl', 'rb') as f:
    y_test = pickle.load(f)

encoder = LabelEncoder()
encoder.fit(np.unique(y_train))
train_labels = encoder.transform(y_train)
val_labels = encoder.transform(y_val)
test_labels = encoder.transform(y_test)
num_classes = len(encoder.classes_)
train_one_hot = keras.utils.to_categorical(train_labels, num_classes=num_classes)
val_one_hot = keras.utils.to_categorical(val_labels, num_classes=num_classes)
test_one_hot = keras.utils.to_categorical(test_labels, num_classes=num_classes)

with open(r'.\embeddingMatrix_News.pkl', 'rb') as f:
    embedding_matrix = pickle.load(f)

num_tokens = len(embedding_matrix) # total vocabulary +1 or length of embedding matrix
embedding_dim = 300 # dimension of the vector of a single word
MAX_NEWS_LEN = 500 # maximum words in a review

In [3]:
def build_model(hp, min_layers, max_layers, test_optimizers, test_activations,
                units_min_value, units_max_value, units_step):
    embedding_layer = keras.layers.Embedding(
        num_tokens,
        embedding_dim,
        embeddings_initializer=keras.initializers.Constant(embedding_matrix),
        input_length=MAX_NEWS_LEN,
        trainable=True,
        mask_zero=True)
   
    model = keras.Sequential()
    model.add(embedding_layer)
    
    num_layers = hp.Int('num_layers', min_layers, max_layers)

    if test_activations:
            activation = hp.Choice(f'activation', ['softplus', 'softsign', 'relu', 'tanh'])
    else:
        activation = 'tanh'  # Default activation

    for i in range(num_layers):
        units = hp.Int(f'units_{i}', min_value=units_min_value, max_value=units_max_value, step=units_step)

        if i < num_layers - 1:  # For all layers except the last one
            model.add(keras.layers.LSTM(units, activation=activation, return_sequences=True))
        else:  # For the last LSTM layer
            model.add(keras.layers.LSTM(units, activation=activation))

    model.add(layers.Dense(num_classes, activation='softmax'))
    
    # Optimizer
    if test_optimizers:
        optimizer = hp.Choice('optimizer', ['SGD', 'RMSprop', 'Adam', 'Adadelta', 'Adagrad', 'Adamax', 'Nadam'])
    else:
        optimizer = 'adam'
    
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [4]:
NUM_EPOCHS = 15
BATCH_SIZE = 128

now = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
directory = f'KerasTuner_Logs/RNN/RNN_V1_3Layer_Density_Optimization_{now}'

# Callbacks
tensorboard = TensorBoard(log_dir=f'TensorBoard_Logs/RNN/RNN_V1_3Layer_Density_Optimization_{now}')
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True, verbose=1)

tuner = kt.GridSearch(
    lambda hp: build_model(hp, min_layers=3, max_layers=3, test_optimizers=False, test_activations=False, 
                           units_min_value=64, units_max_value=128, units_step=32),
    objective=kt.Objective("val_loss", direction="min"),
    max_trials=None,
    executions_per_trial=1,
    directory=directory,
    project_name='Reviews_Classification')

In [5]:
tuner.search(x=X_train,
             y=train_one_hot,
             verbose=1,
             epochs=NUM_EPOCHS,
             batch_size=BATCH_SIZE,
             callbacks=[tensorboard, early_stopping],
             validation_data=(X_val, val_one_hot))

Trial 27 Complete [00h 23m 41s]
val_loss: 0.5366922616958618

Best val_loss So Far: 0.5204403400421143
Total elapsed time: 10h 41m 53s


In [6]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 3, 'units_0': 96, 'units_1': 96, 'units_2': 128}

In [10]:
tuner.get_best_models()[0].summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 500, 300)          145070400 
                                                                 
 lstm (LSTM)                 (None, 500, 128)          219648    
                                                                 
 lstm_1 (LSTM)               (None, 128)               131584    
                                                                 
 dense (Dense)               (None, 10)                1290      
                                                                 
Total params: 145,422,922
Trainable params: 145,422,922
Non-trainable params: 0
_________________________________________________________________


In [21]:
tuner.results_summary()

Results summary
Results in KerasTuner_Logs/RNN/RNN_V1_2Layer_Density_Optimization_2024-07-16_15-45-03\Reviews_Classification
Showing 10 best trials
Objective(name="val_loss", direction="min")

Trial 0000 summary
Hyperparameters:
num_layers: 2
units_{i}: 64
Score: 1.0833885669708252

Trial 0002 summary
Hyperparameters:
num_layers: 2
units_{i}: 128
Score: 1.0934594869613647

Trial 0001 summary
Hyperparameters:
num_layers: 2
units_{i}: 96
Score: 1.0941708087921143
